In [79]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from textblob import TextBlob
import lyricsgenius
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [75]:
client_credentials_manager = SpotifyClientCredentials(client_id='c8ccd1c565bb4056a57b81ea726fd691', client_secret='22c41eabd82f4263bae26f012955b6f6')
scope = "user-read-recently-played"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id='c8ccd1c565bb4056a57b81ea726fd691', client_secret='22c41eabd82f4263bae26f012955b6f6', redirect_uri='http://localhost/'))

#client_credentials_manager = SpotifyClientCredentials(client_id='c8ccd1c565bb4056a57b81ea726fd691', client_secret='22c41eabd82f4263bae26f012955b6f6')
#sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

genius = lyricsgenius.Genius('cJJDWpZwnLSPHD3Yrp6nKSJAS8Ct6da1NDLQEvyZoz1NAIeABj9fmjgi8COuQI2d')


In [76]:
#top_tracks = sp.current_user_top_tracks(limit=50, time_range='medium_term')
#user_input = input("How are you feeling today? ")

#sentiment = TextBlob(user_input).sentiment.polarity
#print(sentiment)

results = sp.current_user_recently_played(limit=10)

In [73]:
recent_tracks = [item['track']['name'] for item in results['items']]
recent_lyrics = []
for track in recent_tracks:
    song = genius.search_song(track)
    if song is not None:
        recent_lyrics.append(song.lyrics)
recent_lyrics_string = ' '.join(recent_lyrics)
#recent_tracks_string = ' '.join(recent_tracks)
sentiment = TextBlob(recent_lyrics_string).sentiment.polarity
print(sentiment)

Searching for "Inside Their Lair"...
Done.
Searching for "Smedjan"...
Done.
-0.1051761696781228


In [95]:
if sentiment > 0.6:
    emotion = "very happy"
elif sentiment > 0.3:
    emotion = "happy"
elif sentiment > 0:
    emotion = "somewhat happy"
elif sentiment < -0.6:
    emotion = "very sad"
elif sentiment < -0.3:
    emotion = "sad"
elif sentiment < 0:
    emotion = "somewhat sad"
else:
    emotion = "neutral"
print (emotion)

somewhat sad


In [51]:
if emotion == "very happy":
    recommended_track = sp.search(q='genre:"happy"', type='track')['tracks']['items'][0]
elif emotion == "happy":
    recommended_track = sp.search(q='genre:"pop"', type='track')['tracks']['items'][0]
elif emotion == "somewhat happy":
    recommended_track = sp.search(q='genre:"dance" ', type='track')['tracks']['items'][0]
elif emotion == "very sad":
    recommended_track = sp.search(q='genre:"sad" ', type='track')['tracks']['items'][0]
elif emotion == "sad":
    recommended_track = sp.search(q='genre:"acoustic"', type='track')['tracks']['items'][0]
elif emotion == "somewhat sad":
    recommended_track = sp.search(q='genre:"alternative" ', type='track')['tracks']['items'][0]
else:
    recommended_track = sp.search(q='genre:"pop"', type='track')['tracks']['items'][0]

# print the recommended track information
print(f"Based on your recent emotions, we recommend: {recommended_track['name']} by {recommended_track['artists'][0]['name']}")

Based on your recent emotions, we recommend: Miss You by Oliver Tree


From here it is just training. Do it if required. Refer to the note pad file i sent for complete code

In [86]:
data = []
target = []

# get tracks with happy emotion
happy_tracks = sp.search(q='genre:"happy" year:2020', type='track', limit=5)['tracks']['items']
for track in happy_tracks:
    song = genius.search_song(track['name'], track['artists'][0]['name'])
    if song is not None:
        data.append(song.lyrics)
        target.append("happy")

Searching for "DLMD" by Darren Styles...
Done.
Searching for "You Are Mine" by S3RL...
Done.
Searching for "FCK 2020" by Scooter...
Done.
Searching for "Paul Is Dead" by Scooter...
Done.
Searching for "DYSYLM" by Gammer...
Done.


In [89]:
pop_tracks = sp.search(q='genre:"pop"', type='track' , limit=5)['tracks']['items']
for track in pop_tracks:
    song = genius.search_song(track['name'], track['artists'][0]['name'])
    if song is not None:
        data.append(song.lyrics)
        target.append("pop")

Searching for "Flowers" by Miley Cyrus...
Done.
Searching for "Sure Thing" by Miguel...
Done.
Searching for "Miracle (with Ellie Goulding)" by Calvin Harris...
Done.
Searching for "Anti-Hero" by Taylor Swift...
Done.
Searching for "ceilings" by Lizzy McAlpine...
Done.


In [90]:
dance_tracks = sp.search(q='genre:"dance"', type='track' , limit=5)['tracks']['items']
for track in dance_tracks:
    song = genius.search_song(track['name'], track['artists'][0]['name'])
    if song is not None:
        data.append(song.lyrics)
        target.append("dance")

Searching for "Escapism." by RAYE...
Done.
Searching for "Sure Thing" by Miguel...
Done.
Searching for "I'm Good (Blue)" by David Guetta...
Done.
Searching for "Miracle (with Ellie Goulding)" by Calvin Harris...
Done.
Searching for "10:35" by Tiësto...
Done.


In [91]:
sad_tracks = sp.search(q='genre:"sad"', type='track', limit=5)['tracks']['items']
for track in sad_tracks:
    song = genius.search_song(track['name'], track['artists'][0]['name'])
    if song is not None:
        data.append(song.lyrics)
        target.append("sad")

Searching for "death bed (coffee for your head)" by Powfu...
Done.
Searching for "Stay With Me" by 1nonly...
Done.
Searching for "Jugaste y Sufrí" by Eslabon Armado...
Done.
Searching for "MARCELINE" by Lil God Dan...
Done.
Searching for "By the Sword" by iamjakehill...
Done.


In [93]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)
y = target
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X, y)

RandomForestClassifier(random_state=42)

In [99]:
if emotion == "somewhat sad":
    happy_songs = sp.search(q='genre:"happy" year:2022', type='track', limit=1)['tracks']['items']
    happy_lyrics = []
    for song in happy_songs:
        try:
            lyrics = genius.search_song(song['name'], song['artists'][0]['name']).lyrics
            happy_lyrics.append(lyrics)
        except lyricsgenius.exceptions.GeniusError:
            print(f"Error: Failed to get lyrics for {song['name']}")
    happy_lyrics_string = ' '.join(happy_lyrics)
    happy_vector = vectorizer.transform([happy_lyrics_string])
    happy_prediction = clf.predict(happy_vector)[0]
    print(f"Recommended song: {happy_songs[0]['name']} by {happy_songs[0]['artists'][0]['name']} (genre: {happy_prediction})")


Searching for "I Follow Rivers (Deep Sea Baby)" by Darren Styles...
Done.
Recommended song: I Follow Rivers (Deep Sea Baby) by Darren Styles (genre: dance)
